In [10]:
import numpy as np
import os

import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from thundersvm import SVC
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder

from pymongo import MongoClient

import sys
sys.path.append('../..')

import utils.dbUtils
import utils.gensimUtils

In [21]:
corpus = utils.dbUtils.TokenizedIterator('news_cleaned', filters = {'type' : {'$in' : ['fake', 'reliable']}, 'domain' : {'$nin' : ['nytimes.com', 'beforeitsnews.com']}}, limit = 25000)

In [22]:
y = np.array([x for x in corpus.iterTags()])
le = LabelEncoder()
le.fit(y)
y = le.transform(y)

In [ ]:
train_accuracy = []
test_accuracy = []
kf = KFold(n_splits=3, shuffle = True)
for i, (train_index, test_index) in enumerate(kf.split(y)):
    print("Iteration {}".format(i))
    model = SVC()
    vectorizer = TfidfVectorizer()
    X_train = vectorizer.fit_transform([' '.join(corpus[i]) for i in train_index])
    X_test = vectorizer.transform([' '.join(corpus[i]) for i in test_index])
    y_train = y[train_index]
    y_test = y[test_index]
    model.fit(X_train, y_train)
    train_accuracy.append(model.score(X_train, y_train))
    test_accuracy.append(model.score(X_test, y_test))
    #print("Training accuracy : {}".format(model.score(X_train, y_train)))
    #print("Test accuracy : {}".format(model.score(X_test, y_test)))
    #print("Classification report for test set")
    #print(classification_report(y_test, model.predict(X_test)))
print("Train accuracy : {}".format(np.mean(train_accuracy)))
print("Test accuracy : {}".format(np.mean(test_accuracy)))
print(test_accuracy)

Iteration 0


In [17]:
X_test, y_test = utils.dbUtils.getFakeNewsNet()

In [26]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform([' '.join(i) for i in corpus])

In [ ]:
model = SVC()
model.fit(X, y)

/home/simon/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [ ]:
X_test = vectorizer.transform(X_test)
print("Test accuracy : {}".format(model.score(X_test, y_test)))

In [ ]:
classification_report(y_test, model.predict(X_test), labels=['fake', 'reliable'], output_dict = True)